## Library

In [15]:
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials
)
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent,AgentType,initialize_agent,create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import BaseTool
from dotenv import load_dotenv
load_dotenv()

from langchain_core.tools import tool,Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools import DuckDuckGoSearchRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import json


python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 6


## Import Model And Model API

In [5]:
model= ChatGoogleGenerativeAI(model="gemini-2.0-flash")
browser_search=DuckDuckGoSearchRun()
wiki_search=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

## Agent As Tool

In [6]:
import re
def clean_report(text):
    
         text = re.sub(r'## ?', '', text)
         text = re.sub(r'\*\*', '', text)
         text = re.sub(r'^\*\s*', '', text, flags=re.MULTILINE)
         text = re.sub(r'^\*+', '', text, flags=re.MULTILINE)
         text = re.sub(r'\*+$', '', text, flags=re.MULTILINE)
         text = re.sub(r'^\*\s*', '', text, flags=re.MULTILINE)
         text = re.sub(r' +', ' ', text)
         return text.strip()

In [ ]:
class email_agent(BaseTool):
	name: str = "email_assistance_tool"
	description: str = "A tool that uses a agent to to create draft email and send email "
	  
	  
	def _run(self, input, **kwargs):
		return self.report_agent(input)
	
	
	
	def report_agent(self, input):
		credentials = get_gmail_credentials(
			token_file="token.json",
			scopes=["https://mail.google.com/"],
			client_secrets_file="Credentials.json",
		)
		api_resource = build_resource_service(credentials=credentials)
		toolkit = GmailToolkit(api_resource=api_resource)
		tools = toolkit.get_tools()
		instructions = """You are an assistant in making report and email ."""
		base_prompt = hub.pull("langchain-ai/openai-functions-template")
		prompt = base_prompt.partial(instructions=instructions)
		agent = create_openai_functions_agent(model, tools, prompt)
		agent_executor = AgentExecutor(
			agent=agent,
			tools=toolkit.get_tools(),
			verbose=False, 
		)
		result = agent_executor.invoke({"input": input})
		return result



class report_agent(BaseTool):
	name: str = "report_generater_tool"
	description: str = "A tool that uses a agent to generate the report By user given topic"
	
	def _run(self, input, **kwargs):
		return self.report_agent(input)
	

	def report_agent(self, input):
		report_template = PromptTemplate(
			input_variables=["topic", "points"],
			template="""Write a professional report about the topic: "{topic}".Here are the main points:{points}
				Include:
				- Title
				- Introduction
				- Key Details
				- Conclusion"""
		)
		report_chain = LLMChain(llm=model, prompt=report_template)
		generate_report_tool = Tool(
			name="GenerateReport",
			func=lambda x: report_chain.run(**json.loads(x)),
			description="Generates a structured report. Input must be JSON with keys 'topic' and 'points'." 
		)
		tools = [generate_report_tool, wiki_search]
		agent = initialize_agent(
			tools=tools,
			llm=model,
			agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
			verbose=True
		)
		report = agent.invoke(input)
		# cleaned_report = clean_report(report["output"])
		return report["output"]


In [8]:
credentials = get_gmail_credentials(
			token_file="token_c.json",
			scopes=["https://mail.google.com/"],
			client_secrets_file="Credentials.json",
		)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=188681790880-gv957ia7tsaf47tdmecs372e9pi65hrp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53547%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=HmBJ5ML1Sf9jVQLkahWDyiLzd8fwJM&access_type=offline


##  Combine Tool


In [18]:
report_generater_tool = report_agent()
email_assistance_tool = email_agent()
agent_tool = [report_generater_tool, email_assistance_tool]

In [19]:
agent_executor = initialize_agent(
    tools=agent_tool, 
    llm=model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)


## Result and test

In [7]:
# report=agent_executor.invoke("darft an email to be sent by me (sheikh shakeel) to My Team (sheikhupdesk@gmail.com) The subject of the email is fifa final match and in the content write about fifa 2018 final match summary in detail and draf it")
# report["output"]

In [20]:
report=agent_executor.invoke("""Create a today report on 'india win t20 worldcup 2024'. Points are: unbeaten india , way of win jurny, final watch, history.""")
report["output"]



> Entering new AgentExecutor chain...
I need to generate a report about India winning the T20 World Cup 2024, highlighting their unbeaten streak, journey to victory, the final match, and historical significance.

Action: report_generater_tool
Action Input: Generate a report on 'India win T20 World Cup 2024'. Points to cover: unbeaten India, way of win journey, final match details, and historical significance.

> Entering new AgentExecutor chain...
I need to generate a report about India winning the T20 World Cup 2024, covering specific points like their unbeaten run, the journey to victory, final match details, and historical significance. I can use the GenerateReport tool for this.

Action: GenerateReport
Action Input: {"topic": "India win T20 World Cup 2024", "points": ["unbeaten India", "way of win journey", "final match details", "historical significance"]}

C:\Users\sk989\AppData\Local\Temp\ipykernel_16184\2267006184.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  func=lambda x: report_chain.run(**json.loads(x)),



Observation: ## India Clinches T20 World Cup 2024: A Dominant Display

**Introduction:**

The Indian cricket team has etched its name in history once again, securing the ICC T20 World Cup 2024 title in a resounding victory. This triumph, achieved after a period of near misses in major tournaments, marks a significant milestone for Indian cricket and reignites the passion of cricket fans worldwide. This report will detail India's unbeaten journey to the championship, highlighting key moments and providing an analysis of the final match, while also examining the historical significance of this victory.

**Key Details:**

*   **Unbeaten India:** India's campaign was characterized by exceptional consistency and strategic brilliance. From the group stages through the knockout rounds, the team remained undefeated, showcasing their dominance in all aspects of the game. This included convincing wins against formidable opponents, demonstrating both batting firepower and disciplined bowling. Th

'I have generated a report on India winning the T20 World Cup 2024, covering their unbeaten streak, journey to victory, the final match details, and historical significance.'

In [23]:
report=agent_executor.invoke("""Create a today report on 'india win t20 worldcup 2024'.
                                Points are: unbeaten india , way of win jurny, final watch, history.and then draf this report to
                                and then take this report as content and darft an email to be sent by me (sheikh shakeel) 
                                to My Team (sheikhupdesk@gmail.com) 
                                The subject of the email is report topic 
                                and draf it """)
report["output"]



> Entering new AgentExecutor chain...
I need to first generate a report on India's T20 World Cup 2024 win, covering the specified points. Then, I need to draft an email using that report as the content, addressed to my team (sheikhupdesk@gmail.com) from Sheikh Shakeel, with the subject being the report topic.

Action: report_generater_tool
Action Input: topic: India win T20 World Cup 2024; points: unbeaten India, way of win journey, final match, history.

> Entering new AgentExecutor chain...
I need to create a report about India's win in the T20 World Cup 2024, focusing on their unbeaten streak, their journey to victory, the final match, and the historical significance.
Action: GenerateReport
Action Input: {"topic": "India win T20 World Cup 2024", "points": ["unbeaten India", "way of win journey", "final match", "history"]}
Observation: ## India Clinches T20 World Cup 2024: An Unbeaten Triumph

**Introduction:**

The cricketing world witnessed a historic moment on [Date of Final] as

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [ ]:
report=agent_executor.invoke("""Create a report on 'final distinace movie'.
                                Points are: all parts , boxoffice colloction, overall rating, about.and then draf this report to
                                and then take this report as content and darft an email to be sent by me (sheikh shakeel) 
                                to My Team (sheikhupdesk@gmail.com) 
                                The subject of the email is report topic 
                                and draf it """)
report["output"]



> Entering new AgentExecutor chain...
Okay, I understand the request. First, I need to generate the report on the movie 'final destination'. Then, I'll use that report to draft an email to be sent to the specified team with the given subject.

Action: report_generater_tool
Action Input: Topic: 'final destination movie'. Points: all parts, box office collection, overall rating, about.

> Entering new AgentExecutor chain...
I need to gather information about the Final Destination movie franchise, including details about each film, their box office performance, overall ratings, and a general overview of the series. I will use the search tools to collect this information and then format it into a report.
Action: duckduckgo_search
Action Input: "Final Destination movie franchise box office"

DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit